## Learning Representations of Gene Ontology: Embedding Term Descriptors using Large Language Models (LLMs)

The Gene Ontology (GO) is a directed knowledge graph where each node represents a specific GO term, defining a particular aspect of genes and their products. These GO terms are categoryzed into three primary ontologies:

- <font color="grey">***Molecular Function (MF)***</font>: These terms define the **activities** performed by gene products, such as *catalysis* or "*transport*", etc. These functions can be refined by more more specific GO terms, such as "protein kinase activity" within the broader category of "catalysis"


- <font color="grey">***Cellular Component (CC)***</font>: These terms specify the subcellular **location** of gene products, emcompasing compartments like *chloroplast* or *nucleus* , as well as macromolecular complexes like *proteasome* or *ribosome*


- <font color="grey">***Biological Process (BP)***</font>: These terms delineate the biological **pathways** in which gene products are involved, ranging from 'DNA repair' and 'carbohydrate metabolic process' to more overaching processes like *biosynthetic processes*

The GO graph integrates these these three aspects of gene annotation, presenting a **hierarchical** organization with **interconnections denoted by edges representing the relationships between terms**. Consequently, the GO graph can be visualized as three distinct but connected trees. However, bear in mind that the three GO ontologies are *'is a'* disjoint because there is no such relationship between terms from different trees, but other types of connections like *'part of'* and *'regulates'* exist. We also define terms as *parents* and *children* based on their position in the tree that we can loosely call *generations*: terms closer to the root are parents, while those closer to the terminal leaves are child terms. Please check the [Gene Ontology reference](https://geneontology.org/docs/ontology-documentation) for more information about how the GO graph is structured.

The GO graph empower us to classify genes and proteins based on their functions in associated processes ocurring in a specific subcellular compartment, being an essential tool to predict functions of newly discovered genes. One feature often overlooked when deploying the GO graph to assist prediction and classification of gene function is the **textual definition of each GO term**. To harness the power of this valuable GO graph content, we will apply **Natural Language Processing** (**NLP**) tecniques to analyze and interpret these statements.


>**Natural Language Processing (NLP)** is a burgeoning field in machine learning, focusing on techniques for extracting contextualized information from word sequences. It finds applications in diverse areas, including sentiment analysis and text generation for product reviews and chatbot interactions.


In this tutorial, we will use the text definitions of GO terms to fine-tune a pretrained Natural Language Processing (NLP) Large Language Model (LLMs). Fine-tunning is a powerful tecnique in machine learning that consists adapting models pre-trained on extensive textual data from different sources to specific applications. This task is done by keeping the deeper encoded features of the pre-trained model but adjusting the trainable parameters of the input and output layers. To demonstrate the power of this approach, we will conduct an analysis of Molecular Biology articles and try to accuratelly identify which primary ontology (BP, MF or CC) best characterizes the paper. This practical exercise will highlight the practicality of fine-tuning within a real-world context. We will also use the fine-tunned model to embed the text definitions into feature vectors to enrich nodes' features in the GO graph. By adding this additional attribute to the nodes, we aim to improve the power of GO graph in assisting machine learning applications such as prediction of gene functions and the outcome of metabolic pathways.


>**Text Embeddings**:  Machine learning algorithms primarily operate on numerical data. To utilize text data in these models, it must be transformed into numerical representations. Text embeddings are techniques that extract meaningful hidden features from text while preserving the sequential relationships between words. This ensures that the essential meaning of the text is preserved in the feature vector. 


In [ ]:
# install required libraries

# !pip install obonet
# !pip install nltk
# !pip install transformers
# !pip install datasets
# !pip install wordcloud

In [ ]:
# import libraries

import re
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from obonet import read_obo
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, balanced_accuracy_score
from yellowbrick.text import FreqDistVisualizer
from wordcloud import WordCloud
from tqdm.notebook import tqdm_notebook

# progress bar for loops
tqdm_notebook.pandas()

# set home directory
from pathlib import Path

home_dir = Path("../GO-graph-embed/")
print(f"Home directory: {home_dir.as_posix()}")

# download / update punkt, the nltk's recommended sentences' tokenizer
import nltk

nltk.download("punkt")

# custom functions

# plot confusion matrix
def plot_cm(y_true: pd.Series, y_pred: pd.Series):
    # print out the balanced accuracy score
    print(f"Balanced Accuracy Score: {balanced_accuracy_score(y_pred, y_true):.1%}")

    cm = confusion_matrix(
        y_true,
        y_pred,
        normalize="pred",
    )

    # converting into dataframe for easy plotting
    cm_df = pd.DataFrame(
        cm,
        index=["Biological Process", "Cellular Component", "Molecular Function"],
        columns=["Biological Process", "Cellular Component", "Molecular Function"],
    )

    plt.figure(figsize=(10, 8))
    sns.set(font_scale=1.2)
    sns.heatmap(
        data=cm_df,
        annot=True,
        cmap="Blues",
    )
    plt.title("Confusion Matrix")
    plt.ylabel("True")
    plt.xlabel("Predicted")
    plt.show()

## Reading and Parsing GO Graph Data

### File format

The GO graph is stored in the *OBO (Open Biomedical Ontologies) file oboformat*, which is specifically tailored for constructing and representing biological ontologies. We can use the Python library *obonet* to read and parse the *obo* file to convert it into a **NetworkX** object. [NetworkX](https://networkx.org/) provides a comprehensive framework to manipulate and analyze graphs.

In [ ]:
go_graph = read_obo(home_dir.joinpath("data/go-basic.obo"))

print(f"The GO graph is a {go_graph}.")

### Node attributes

The nodes contain attributes describing the respective GO terms. let's print out the attributes of a randomly selected node to get a glimpse these attributes:

In [ ]:
go_graph.nodes()[random.choice([x for x in go_graph.nodes()])]

All GO terms contain the following essential attributes:

>***name***: unique identifier of the term in a human-readable format

>***namespace***: one of the three major ontologies (MF, CC or BP) to which the term belongs

>***definition***: short description of what the GO term means for humans. It can also contains references to publications defining the term (e.g. PMID:10873824).

There are additional attributes of each node corresponding to GO terms, but we won't be using them in this project.

### Creating a Dataframe with Nodes' attributes

We will extract the attributes from GO graph nodes into a dataframe to facilitate the processing of the text definitions. We will also add a column with the length of the text definition of each GO term. This will help us to determine the parameters when deploying **pre-trained Large Language Models (LLMs)** to extract features from GO terms' definitions and embed them into a feature vector.

In [ ]:
# create GO definitions dataframe

# empty dataframe to store nodes' attributes
go_df = pd.DataFrame(columns=["go_id", "name", "aspect", "definition", "length"])

# iterating over nodes to extract dictionary keys and values
for idx, item in tqdm_notebook(
    enumerate(go_graph.nodes.items()), total=len(go_graph.nodes)
):
    go_term = item[0]
    name = item[1]["name"]
    aspect = item[1]["namespace"]
    definition = item[1]["def"].split(sep='"', maxsplit=2)[1]
    length = len(definition)
    go_df.loc[idx] = [
        go_term,
        name,
        aspect,
        definition,
        length,
    ]

# saving dataframe
home_dir.joinpath("data/").mkdir(parents=True, exist_ok=True)
go_df.to_csv(home_dir.joinpath("data/go_df.csv"), index=False)

go_df

In [ ]:
# loading saved dataframe with GO graph nodes' attributes

go_df = pd.read_csv(home_dir.joinpath("data/go_df.csv"))

print(f"GO entries: {go_df.shape[0]}, features: {go_df.shape[1]}")
display(go_df.head())

We can also visually inspect the length (the **number of characters** in this case...) distribution of GO terms text definitions. The [`seaborn.histplot()`](https://seaborn.pydata.org/generated/seaborn.histplot.html) function can plot the *kernel density distribution* (*KDE*) to estimate the text length distribution:

In [ ]:
go_hist = sns.histplot(go_df["length"].values, bins=100, kde=True)
go_hist.set_title("GO definition lengths")
go_hist.set_xlabel("Definition Lengths")

plt.show()

By plotting the length of the GO terms definitions and analyzing the quartiles, we can see that most definitions fall bellow 200 words (75%), with mean length of ~150 words.

## Pre-Trained Transformer Language Models (LLMs)

[**Transformers**](https://huggingface.co/docs/transformers/index) for text classification is a 'hot' topic in machine learning and behind hipped generative models like [chatGPT](https://chat.openai.com). Text transformer architectures are designed to find dependencies between words in a sentence regarding their positions and use these dependencies for text ranking or classification, in addition to generative models capable of generate new sentences based on a query.

We will **fine-tune** (i.e. train the model to perform a **specific task**) the pretrained **Large Language Model** (LLM) [bert-base-multilingual-cased](https://huggingface.co/bert-base-multilingual-cased) to classify GO term definitions regarding the major GO ontologies (BP, CC and MF) to which each one belong. We will also obtain text embeddings of GO term definitions to use them as additional data to train models to predict protein annotations.

The model is the  based on the [Bidirectional Encoder Representations from Transformers (BERT)](https://huggingface.co/docs/transformers/model_doc/bert) architecture pre-trained with millions of [Wikipedia](https://www.wikipedia.org/) entries in 104 languages.

## Creating Datasets

We need to preprocess the dataset in such a way that the pre-trained model can understand. First we'll create train, validation and test datasets with the input text from GO terms' definitions. Then, the datasets will be **'tokenized'**, i.e., the input texts will be splitted into tokens using a set of rules and delimited by special tokens that says to the model where the sequence and sentences start and end.

 The datasets are created using the Hugging Face's library [Dataset](https://huggingface.co/docs/datasets/index). Actually, later we will be using pre-trained sequence classification models and respective tokenizers from [Hugging Face](https://huggingface.co/).

Before that, we change the *definition* column name to *text* and change the *aspect* column type to categorical:

In [ ]:
# rename definition column
go_df.rename(columns={"definition": "text"}, inplace=True)

# converting categorical labels to numbers (aspect)
go_df["aspect"] = pd.Categorical(go_df["aspect"])

# getting categorical codes
go_df["label"] = go_df["aspect"].cat.codes

display(go_df)
print(go_df.dtypes)

We can iterate over the aspect categories we just created to check the correspondence between the classes and the label numbers:

In [ ]:
for code, aspect in enumerate(go_df.aspect.cat.categories):
    print(f"{code}: {aspect}")

Selecting only the columns we need for model's fine-tunning:

In [ ]:
data = go_df[["text", "label"]].copy()
data

We can visualize the most frequent terms in the data by plotting a *word cloud*, which is a nice way to grasp the most frequent words:

In [ ]:
go_wordcloud = WordCloud(
    width=800, height=800, background_color="white", min_font_size=10, colormap="tab20b",
).generate(data.text.to_string())

plt.figure(figsize=(8, 8), facecolor=None)
plt.suptitle("Word cloud of GO term definitions")
plt.imshow(go_wordcloud)
plt.axis("off")
plt.tight_layout(pad=1)
plt.show()

The number of characters of each definition is showed in the *def_length* column. The language model that we'll fine-tune and use for predictions accepts a limited number of tokens as input In our case study, the **bert-base-multilingual-cased** accepts **512 tokens** (actually 500 tokens and two special tokens [CLS] and [SEP] that we will get to know later).

A box-plot can provide useful insight on the lengths to choose the better hyperparameters for tokenization and training. Here, we use the **re** module to find all the words in each definition and return the length using a **lambda** function applied to the *definition* column:

In [ ]:
plt.figure(figsize=(2, 4))
plt.title("Dataset Word Count")

plt.boxplot(data.text.apply(lambda x: len(re.findall(r"\w+", x))))
plt.ylabel("Word Count")

plt.show()

All GO terms' definitions fit the input size required for fine-tuning (510 tokens) but we may want to limit the length of inputs to speed-up the fine-tuning process. Let's proceed to create the dataset with train and test sets using Hugging Face's Dataset `train_test_split()` method. We use stratification to preserve labels ratios between datasets:

In [ ]:
from datasets import Dataset

# create dataset
dataset = Dataset.from_pandas(data)

# change label column to ClassLabe to allow stratification
dataset = dataset.class_encode_column("label")

dataset = dataset.train_test_split(test_size=0.3, stratify_by_column="label")
dataset

The **Tokenization strategy must match the model of choice**. This is key to ensure that tokens will be mapped to the **same indices** presented to the model during the training phase and also use the **same special tokens** to delimit the beggining of a text sequence and the separation between sentences.

Here, we use [BertTokenizerFast](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertTokenizerFast) using the tokenization strategy of the pre-trained model (bert-base-multilingual-uncased) to get rid of uppercase and capital letters. With lowercase inputs, the number of tokens is usually smaller and generalizes better for unseen text sequences inputs in **production phase**. Moreover, we can change it later and see if the model performs better or not with cased inputs.


In [ ]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, max_length=150, truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

tokenized_dataset

Explaining the keys in train_ds sample items:

- **input_ids**: indices of each token in the sequence (these indices were generated during model pre-training)
- **token_type_ids**:  references of the sentence to which the token belongs
- **attention_mask**: indicates if the token should be attended or not

We can decode the the **input_ids** to obtain the original text. Below we print out the decoded definitions based on the input_ids (I did some changes in the print for better visualization of the decoded input ids and the input ids):

In [ ]:
dataset_len = len(tokenized_dataset["train"]["input_ids"])

for i in range(5):
    idx = random.randint(0, dataset_len)
    print(tokenizer.decode(tokenized_dataset["train"]["input_ids"][idx]))
    input_ids_list = tokenized_dataset["train"]["input_ids"][idx]
    print(input_ids_list)
    print(f"Input length: {len(input_ids_list)}")
    print(50 * "=")

In the output above we can see the sentences delimited by the special tokens **Classification Token [CLS]** and **Separator Token [SEP]**. [CLS] delimits the start of the sequence for the BERT model we will be using. 

and set the model to generate embeddings for sequence classification tasks. [SEP] is used to separate sentences in sentence-pair tasks and help the model to capture the relationships between the two senteces concatenated by the [SEP]. The **padding to ensure same-sized sentences** during tokenization is represented by the special padding token **[PAD]**.

>**Truncation and Padding**: truncation and padding parameters tell to the tokenizer how to handle sequences of variable length. Input sequences with more tokens than the number accepted by the model will be truncated if *truncation* parameter is 'True'. On the other hand, if the sequences are shorter than the model's accept, the special padding token [PAD] will be added until the number of tokens in the input match the model's requireiments. *Truncation* and *padding* are key to ensure that the input will be the same size. This is key because the input of the model are fixed-size tensors set during model training (510 + 2 special tokens [CLS] and [SEP] = 512).

Now, let's instantiate the model class [BertForSequenceClassification](https://huggingface.co/docs/transformers/v4.33.2/en/model_doc/bert#transformers.BertForSequenceClassification) loaded with the **weights** from the pre-trained model "bert-base-multilingual-cased" and define the number of target labels we want to predicted. In our case-study, we will predict if the text talks about Biological Process, Cellular Component or Molecular Function. Thus, the number of labels is **3**.

In [ ]:
import torch

print(f"cuda available?: {torch.cuda.is_available()}")
print(torch.cuda.memory_summary())

In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-uncased", num_labels=3)

model.cuda()
model

We will fine-tune the model using the [Trainer](https://huggingface.co/docs/transformers/v4.33.2/en/main_classes/trainer#transformers.Trainer) class optimized to train Hugging Face's models.

We need to configure the **trainer's hyperparameters** using the [TrainingArguments](https://huggingface.co/docs/transformers/v4.33.2/en/main_classes/trainer#transformers.TrainingArguments) and set the evaluation metrics using the [evaluate](https://huggingface.co/docs/evaluate/index) module to monitor the training process and evaluate the performance after fine-tuning.

>To see the full list of tunable training hyperparameters and available metrics, click the links above. Alternativelly, you can make some elegant code to show them.

In [ ]:
# PyTorch Trainer
from transformers import TrainingArguments
import accelerate

train_args = TrainingArguments(output_dir="test_trainer")

# list training parameters
args_list = [
    x for x in list(dir(train_args)) if not re.compile(r"\b_[A-Za-z0-9_]*\b").match(x)
]
print(f"There are {len(args_list)} training arguments: {args_list}")

Since the Trainer does not have an automatic evaluator of model's performance during training, we have to add metrics using *evaluate* and pass it to Trainer through TrainingArguments.

We can use ROC/AUC score to assess model's performance. Additionally, we need to define a function to convert the predictions to logits to pass it to the *compute* method as explained by [this tutorial](https://huggingface.co/docs/transformers/training) in Hugging Face:

In [ ]:
import evaluate

# list metrics
metrics_list = evaluate.list_evaluation_modules()
print(f"There are {len(metrics_list)} available metrics: {metrics_list}")

# load roc_auc metric
metric = evaluate.load("accuracy")


# convert preds --> logits
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


# set evaluation for training
train_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",
)

In [ ]:
small_train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(10000))
small_eval_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(2000))

print(small_train_dataset)
print(small_eval_dataset)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


trainer.train()

## Fine-Tunning in Native PyTorch

In [ ]:
import torch

del model
del trainer
torch.cuda.empty_cache()